In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import matplotlib as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import re
import gensim
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from gensim.models import CoherenceModel
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer



In [26]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/ayman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ayman/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ayman/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Splitting reviews

In [27]:
df = pd.read_csv('/home/ayman/Documents/Machine learning/html_to_csv.csv')
data = pd.DataFrame(columns=["Reviews","ID"])
for i in range (len(df)):
   listo =[]
   listo =df["Reviews"].loc[i].split("ZABALITE lelda3m el-fanny")[0]
   data.at[i, 'Reviews'] = [listo]
for i in range (len(df)):
   data["ID"] =df["ID"] 
data


,Reviews,ID
0,[['Do the exercises included and you will defi...,492566615
1,"[[""Let me just say this game is fun and great ...",1179624229
2,[['Have been playing this game since 2012! Pla...,500099330
3,[['When buying purchases why doesn’t my stars ...,633807835
4,"[['Pretty addictive game. I loved it', ']",1144606268
...,...,...
2720,"[[""Bar none. No annoying ads, you're simply re...",442239737
2721,[['Good game but the island in the game is sma...,848246447
2722,"[[""It's an ok game but sometimes it won't let ...",643705579
2723,[['Incredibly clever. Will we see voiceover su...,1020657631


### Preprocessing


In [28]:
# This used to work when data only contained one column now when it contains 2 it doesnt

for i in range(len(data)):
    # convert text to lowercase
    data.at[i, 'Reviews'] = [x.lower() for x in data.at[i, 'Reviews']]
    # # replace newline characters with an empty string
    data.at[i, 'Reviews'] = [x.replace('\\n', ' ') for x in data.at[i, 'Reviews']]
    # # remove black squares with
    data.at[i, 'Reviews'] = [x.replace('\\u25a0', '') for x in data.at[i, 'Reviews']]
    # # remove special characters and punctuations
    data.at[i, 'Reviews'] = [re.sub(r'[^\w\s]+', '', x) for x in data.at[i, 'Reviews']]
    # # remove numbers
    # # descriptions['Description'][i] = re.sub(r'\d', '', descriptions['Description'][i])
    data.at[i, 'Reviews'] = [" ".join([word for word in x.split() if not any(char.isdigit() for char in word)]) for x in data.at[i, 'Reviews']]
    # # remove extra whitespaces
    data.at[i, 'Reviews'] = [re.sub(r'\s+', ' ', x) for x in data.at[i, 'Reviews']]
    # # remove stop words
    data.at[i, 'Reviews'] = [" ".join([word for word in x.split() if word not in stop_words])for x in data.at[i, 'Reviews']]
data


,Reviews,ID
0,[exercises included definitely improve levelal...,492566615
1,[let say game fun great especially bored there...,1179624229
2,[playing game since played nonstop years ever ...,500099330
3,[buying purchases doesnt stars immediately app...,633807835
4,[pretty addictive game loved],1144606268
...,...,...
2720,[bar none annoying ads youre simply restricted...,442239737
2721,[good game island game smaller free version co...,848246447
2722,[ok game sometimes wont let swing game ok favo...,643705579
2723,[incredibly clever see voiceover support],1020657631


### Tokenization & lemmatization

In [29]:
# Tokenization
for i in range(len(data)):
    data.at[i, 'Reviews'] = [word_tokenize(desc) for desc in data.at[i, 'Reviews']]
# Lemmatization
lemmatizer = WordNetLemmatizer()
data['Reviews'] = data['Reviews'].apply(lambda x: [[lemmatizer.lemmatize(word) for word in sent] for sent in x])
data


,Reviews,ID
0,"[[exercise, included, definitely, improve, lev...",492566615
1,"[[let, say, game, fun, great, especially, bore...",1179624229
2,"[[playing, game, since, played, nonstop, year,...",500099330
3,"[[buying, purchase, doesnt, star, immediately,...",633807835
4,"[[pretty, addictive, game, loved]]",1144606268
...,...,...
2720,"[[bar, none, annoying, ad, youre, simply, rest...",442239737
2721,"[[good, game, island, game, smaller, free, ver...",848246447
2722,"[[ok, game, sometimes, wont, let, swing, game,...",643705579
2723,"[[incredibly, clever, see, voiceover, support]]",1020657631
